# Workshop 3: Explore NT scenario results, modify assumptions & benchmarking

:::{note} At the end of this notebook, you will be able to:

- Navigate and analyze NT scenario results
- Interpret current discrepancies between model outputs and the TYNDP 2024 NT scenario
- Compare 2030 and 2040 scenarios using PyPSA-Explorer
- Modify model assumptions and generate new scenario results
- Apply the benchmarking framework to identify areas for model improvement

:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pypsa-explorer pandas matplotlib numpy
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa pypsa-explorer pandas matplotlib numpy

In [ ]:
import os
from datetime import datetime
import pandas as pd
import pypsa
import zipfile
from urllib.request import urlretrieve
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError
from IPython.display import Code, display
import matplotlib.pyplot as plt
from pypsa_explorer import create_app
from pathlib import Path

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False
plt.rcParams["figure.figsize"] = [14, 7]

In [ ]:
def unzip_with_timestamps(zip_path, extract_to):
    """Unzip a file while preserving original file timestamps."""
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for member in zip_ref.infolist():
            # Extract the file
            zip_ref.extract(member, extract_to)

            # Get the extracted file path
            extracted_path = os.path.join(extract_to, member.filename)

            # Get the modification time from the zip file
            date_time = datetime(*member.date_time)
            timestamp = date_time.timestamp()

            # Set both access and modification times
            os.utime(extracted_path, (timestamp, timestamp))

In [ ]:
urls = {
    "data/20251127_results.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-03/20251127_results.zip",
    "data/open-tyndp.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-02/open-tyndp.zip",
    "scripts/_helpers.py": "https://raw.githubusercontent.com/open-energy-transition/open-tyndp-workshops/refs/heads/feature/workshop-03-notebook/open-tyndp-workshops/scripts/_helpers.py",  # TODO: Update for version on main
}

os.makedirs("data", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/20251127_results"
if not os.path.exists(to_dir):
    print(f"Unzipping data/20251127_results.zip.")
    unzip_with_timestamps("data/20251127_results.zip", "data/20251127_results")
print(f"NT results available in '{to_dir}'.")

to_dir = "data/open-tyndp"
if not os.path.exists(to_dir):
    print(f"Unzipping data/open-tyndp.zip.")
    unzip_with_timestamps("data/open-tyndp.zip", "data/open-tyndp")
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# Interactive exploration with PyPSA-Explorer

PyPSA-Explorer is an interactive web dashboard that allows you to visualize and analyze energy system networks. It provides:
- Energy balance analysis with timeseries and aggregated views
- Capacity planning visualizations by carrier and region
- Economic analysis showing CAPEX/OPEX breakdowns
- Interactive geographical network maps
- Multi-network visualisation support

Let's load the NT scenario results and explore them using PyPSA-Explorer.

In [ ]:
# Load NT scenario networks for comparison
base_path = "data/20251127_results/networks/"

# Load networks directly into dictionary for PyPSA-Explorer
networks = {
    "NT 2030": pypsa.Network(base_path + "base_s_all___2030.nc"),
    "NT 2040": pypsa.Network(base_path + "base_s_all___2040.nc"),
}

The PyPSA-Explorer package can be started in two modes depending on your environment:

- **Local Jupyter**: Terminal command (**recommended**) or inline display
- **Google Colab**: The dashboard will launch inline, embedded directly in the notebook

Follow the instructions below based on your environment.

In [ ]:
# Detect if running on Google Colab
try:
    from google.colab import output

    IN_COLAB = True
    print(f"This notebook is running on Google Colab !")
except ImportError:
    IN_COLAB = False
    print(f"This notebook is running locally !")

port = 8050

## For Local Users

If running locally, the **recommended approach** is to launch PyPSA-Explorer from the terminal:

```bash
pypsa-explorer data/20251127_results/networks/base_s_all___2030.nc:NT_2030 data/20251127_results/networks/base_s_all___2040.nc:NT_2040
```

This will open the dashboard in your default browser at http://localhost:8050.

**Alternative**: The cell below can also launch the dashboard inline, but the terminal method is preferred for better performance.

In [ ]:
# Terminal method recommended
USE_TERMINAL = True  # Change to False if you want to launch inline display

if not IN_COLAB and not USE_TERMINAL:
    # Local Jupyter: Inline display
    app = create_app(networks)
    app.run(jupyter_mode="tab", port=port, debug=False)

## For Google Colab Users

If running on Google Colab, you need to start PyPSA-Explorer from the terminal before the iframe can display it.

Run the following command in the Google Colab terminal:

```python
pypsa-explorer data/20251127_results/networks/base_s_all___2030.nc:NT_2030 data/20251127_results/networks/base_s_all___2040.nc:NT_2040
```

In [ ]:
if IN_COLAB:
    # Google Colab: Use built-in Dash Jupyter support to display an iframe
    output.serve_kernel_port_as_iframe(port, height=1500)

**Tip for Colab users:** To view the dashboard in fullscreen mode, click the three dots (⋮) in the top-right corner of the output cell and select **"View output fullscreen"**.

## Using the Dashboard

Once the dashboard opens, you can:

1. **Energy Balance Tab**: View production, consumption, and storage patterns
   - Switch between timeseries and aggregated views
   - Filter by energy carrier (electricity, hydrogen, etc.)
   - Filter by country/region

2. **Capacity Tab**: Analyze installed capacities
   - Compare capacity buildout between 2030 and 2040 scenarios
   - View by technology type and region
   - Identify discrepancies with TYNDP 2024 targets

3. **Economics Tab**: Examine costs and revenues
   - CAPEX and OPEX breakdowns by technology
   - Regional cost comparisons
   - Investment requirements

4. **Network Map**: Visualize the geographical network
   - Interactive map with buses, lines, and generators
   - Zoom and pan to explore specific regions
   - View transmission constraints

**Tip:** Use the scenario selector dropdown to switch between NT 2030 and NT 2040 for direct comparison.

# Benchmark results

In Workshop 2, we introduced a benchmarking framework to systematically compare Open-TYNDP model outputs against TYNDP 2024. This framework helps us:

- **Identify discrepancies**: Compare demands, installed capacities and generation volumes between model results and TYNDP targets
- **Quantify differences**: Calculate deviations by technology, country, and investment year
- **Guide improvements**: Prioritize areas where the model requires improvements

Let's apply this framework to our NT 2030 and 2040 scenario results to understand where the current model implementation aligns with or diverges from TYNDP expectations.

To do so, let's first define a function to display benchmarks.

In [ ]:
def show_benchmarks(
    fn: str,
    years: list = [2030, 2040],
    bench_path: str = "data/20251127_results/validation/graphics_s_all___all_years",
):
    try:
        images = [
            convert_from_path(Path(bench_path, f"{fn}_{y}.pdf"))[0] for y in years
        ]
    except PDFPageCountError:
        print("File not found, skipping...")
        return

    fig, axes = plt.subplots(1, 2)
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

## Overview

We start with an overview of the sMAPE across all carriers. This provides a high-level understanding of the magnitude of the error across all benchmarks, carriers and planning horizons. It can clearly be observed that some features were significantly improved since our introduction of the framework. However, there is also still room for improvement.

In [ ]:
display(
    convert_from_path(
        Path("data/20251127_results/validation/kpis_eu27_s_all___all_years.pdf")
    )[0]
)

## Final Energy Demand

Let's now examine each benchmark category in detail, starting with final energy demand across all carriers.

In [ ]:
show_benchmarks("benchmark_final_energy_demand_eu27_cy2009")

## Electricity Demand

Electricity demand is an exogenous input to the model, directly taken from TYNDP 2024 data. Therefore, we get a perfect match between the model results and TYNDP 2024.

In [ ]:
show_benchmarks("benchmark_elec_demand_eu27_cy2009")

## Hydrogen Demand

Hydrogen demand is mainly defined exogenously. Therefore, we expect a good match between the model results and TYNDP 2024. The electricity generation, on the other hand, is endogenous.

In [ ]:
show_benchmarks("benchmark_hydrogen_demand_eu27_cy2009")

## Methane Demand

Methane demand includes exogenous consumption, as well as endogenous demand for power generation and SMR (steam methane reforming).

In [ ]:
show_benchmarks("benchmark_methane_demand_eu27_cy2009")

## Power Generation Capacities

Installed generation capacities are now converging towards the values of TYNDP 2024. Both renewable and conventional capacities are now fully integrated from the PEMMDB.

In [ ]:
show_benchmarks("benchmark_power_capacity_eu27_cy2009")

## Electricity Generation

Actual electricity generation differs slightly from the TYNDP 2024 generation mix. However, the total generation values are close.

In [ ]:
show_benchmarks("benchmark_power_generation_eu27_cy2009")

## Hydrogen Supply

TODO

In [ ]:
show_benchmarks("benchmark_hydrogen_supply_eu27_cy2009")

## Methane Supply

TODO

In [ ]:
show_benchmarks("benchmark_methane_supply_eu27_cy2009")

## Biomass Supply

TODO

In [ ]:
show_benchmarks("benchmark_biomass_supply_eu27_cy2009")

## Energy Imports

Energy imports include hydrogen, methane, liquids and solids. Currently, the model cannot import biomass.

In [ ]:
show_benchmarks("benchmark_energy_imports_eu27_cy2009")

# Modify assumptions

A key requirement when working with any energy system model is to adjust input assumptions and see how results respond. This is especially true for TYNDP models, where assumptions evolve over time and many configurations need to be tested and re-run repeatedly.

In Open-TYNDP, there are several straightforward ways to change assumptions and explore scenarios. In practice, you’ll mostly use one (or a mix) of these options:
- **Input data:** Update the relevant files in the `data` folder.  
- **Custom assumptions:** Override cost and technology assumptions via `custom_cost.csv`.  
- **Adjustments:** Make targeted changes to specific components through `scenario.config.yaml`.
- **Custom constraints:** Add or adjust constraints directly in the `solve_network.py` script.

## Modifying the input data

The first way to make changes to the model is by editing the input data directly.

To get started, navigate to the `data` directory in the open-tyndp repository and locate the `tyndp_2024_bundle` folder.

In [ ]:
from scripts._helpers import display_tree

target_directory = "data/open-tyndp/data/tyndp_2024_bundle"
print("data")
display_tree(target_directory, max_depth=1)

Here you can freely browse the input files and replace any of them with your own inputs, as long as it follows the **same format**.

## Custom assumptions

If you’d like to add custom assumptions, for example on capital costs, marginal costs, or technical parameters for specific technologies, you can do so in a long-format CSV table called `custom_cost.csv` in the `data` folder.

So, head back to the `data` directory and open `custom_cost.csv`. You’ll notice that the file already contains a few example assumptions to illustrate the structure:

In [ ]:
custom_cost = pd.read_csv("data/open-tyndp/data/custom_costs.csv")
custom_cost

If you want to add your own custom assumptions, simply insert a new row in this CSV. At a minimum, each row should specify the `planning_horizon`, `technology`, `parameter`, `value`, and `unit`.

These entries will automatically be picked up the when you rerun the Open-TYNDP workflow.

## Adjustments

The third way to modify Open-TYNDP assumptions is by making targeted changes to specific technologies in the system.

These are called `adjustments`, and you add them directly in the model configuration file for your scenarios.

You can find an example of this section in the default configuration file:

In [ ]:
def display_code_lines(filename, language, start, end):
    with open(filename) as f:
        lines = f.readlines()
    return Code("".join(lines[start - 1 : end]), language=language)

In [ ]:
display_code_lines("data/open-tyndp/config/config.default.yaml", "yaml", 1069, 1076)

As you can see, it is possible to specify either a scaling factor or an absolute value for a given:
- component type
- carrier (i.e. technology) and
- attribute

So, if you want to make a manual adjustment to a scenario you’re modelling, you can do that by adding an `adjustments` section to your `scenarios.tyndp.yaml`.

Let’s open that file and take a look:

In [ ]:
display_code_lines("data/open-tyndp/config/scenarios.tyndp.yaml", "yaml", 1, 100)

## Custom constraints

Lastly, you might want to add custom constraints to the model. Since we haven’t covered PyPSA constraints yet, we won’t go into detail here, but we’ll treat them comprehensively in one of our next workshops.

Still, it’s useful to know where you *would* add a custom constraint that isn’t already represented through existing component parameters.

**Note:** An example for a standard PyPSA constraint is the `p_nom_max` attribute of a `Link` component. PyPSA automatically translates this into a binding upper-limit constraint on the capacity of an expandable `Link`.

To add a custom constraint, you can insert your own code directly into the model. This kind of flexibility is one of the big advantages of working with an open-source setup.

Please navigate to the `scripts` directory of the`open-tyndp` repository and open the `solve_network.py` script.

In [ ]:
display_code_lines("data/open-tyndp/scripts/solve_network.py", "Python", 1268, 1361)

In `solve_network.py`, you’ll find a function called `extra_functionality`. This is where additional custom constraints are added to the model.

For instance, Open-TYNDP includes a custom constraint for the TYNDP offshore hubs by calling the nested function `add_offshore_hubs_constraint`.

You can add your own constraint in the same place. Feel free to browse the constraints already implemented there to get a sense of the structure and style.

We’ll cover how PyPSA constraints work in more depth in one of the next workshops.

## Task 1: Add manual adjustments

Modify the existing `NT` scenario in `scenario.tyndp.yaml` by adding the following manual adjustments as explained before:

- Increase the `marginal_cost` of all H2 imports by a factor of 1.5 (2030) and 1.3 (2040) (The supply of import H2 is included as a `Generator` component with the common carrier name `import H2`).
- Change the `efficiency` of `H2 Electrolysis` to 78% for both years 2030 and 2040 (H2 Electrolysis is added as a `Link` component).
- Remove the initial capacity (`p_nom` and `p_nom_min`) of all `solar-pv-utility` generators for 2030.

Then rerun the model and explore the results in the PyPSA-Explorer.

**Hint:** If you need a reminder on running the Snakemake workflow, check the notebook from our last workshop.  
**Hint II:** Always start with a dry run first (add `-n` at the end of your Snakemake command).

# Solutions

## Task 1: Add manual adjustments

The following section needs to be added to the `NT` scenario configuration in `scenario.tyndp.yaml`:
```yaml
adjustments:
  sector:
    factor:
      Generator:
        import H2:
          marginal_cost:
            2030: 1.5
            2040: 1.3
        solar-pv-utility:
          p_nom:
            2030: 0.0
    absolute:
      Link:
        H2 Electrolysis:
          efficiency:
            2030: 0.78
            2040: 0.78
```

As we learned last time, we first need to navigate into our open-tyndp repository to launch the workflow:

In [ ]:
os.chdir("data/open-tyndp")

Let's check that we are indeed in the new directory now:

In [ ]:
os.getcwd()

Having added our adjustment to the `scenario.tyndp.yaml`, we can now use Snakemake to call the `open-tyndp` workflow again using the standard `config.tyndp.yaml` configuration file.

First, a dry-run...

In [ ]:
! snakemake -call --configfile config/config.tyndp.yaml -n

As you can see, because this scenario has been run before without the adjustments, Snakemake will only re-execute the rules that are affected by our manual adjustments.

Now, let's actually execute the workflow.

In [ ]:
# ! snakemake -call --configfile config/config.tyndp.yaml